<a href="https://colab.research.google.com/github/Mohan6677/ml_zoomcamp/blob/main/HW_WK3_MLZOOMCAMP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [403]:
import pandas as pd
import numpy as np

import seaborn as sns
from matplotlib import pyplot as plt
%matplotlib inline

In [404]:
df = pd.read_csv('bank-full.csv', sep=';')

In [405]:
df.head()

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,58,management,married,tertiary,no,2143,yes,no,unknown,5,may,261,1,-1,0,unknown,no
1,44,technician,single,secondary,no,29,yes,no,unknown,5,may,151,1,-1,0,unknown,no
2,33,entrepreneur,married,secondary,no,2,yes,yes,unknown,5,may,76,1,-1,0,unknown,no
3,47,blue-collar,married,unknown,no,1506,yes,no,unknown,5,may,92,1,-1,0,unknown,no
4,33,unknown,single,unknown,no,1,no,no,unknown,5,may,198,1,-1,0,unknown,no


In [406]:
df.dtypes

,0
age,int64
job,object
marital,object
education,object
default,object
balance,int64
housing,object
loan,object
contact,object
day,int64


In [407]:
required_columns = ['age', 'job', 'marital', 'education', 'balance', 'housing',
                    'contact', 'day', 'month', 'duration', 'campaign', 'pdays',
                    'previous', 'poutcome', 'y']

In [408]:
df.columns = df.columns.str.lower().str.replace('"', '')
df.dtypes

,0
age,int64
job,object
marital,object
education,object
default,object
balance,int64
housing,object
loan,object
contact,object
day,int64


In [409]:
df = df[required_columns]
df.head()

,age,job,marital,education,balance,housing,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,58,management,married,tertiary,2143,yes,unknown,5,may,261,1,-1,0,unknown,no
1,44,technician,single,secondary,29,yes,unknown,5,may,151,1,-1,0,unknown,no
2,33,entrepreneur,married,secondary,2,yes,unknown,5,may,76,1,-1,0,unknown,no
3,47,blue-collar,married,unknown,1506,yes,unknown,5,may,92,1,-1,0,unknown,no
4,33,unknown,single,unknown,1,no,unknown,5,may,198,1,-1,0,unknown,no


In [410]:
df['education'].describe()

,education
count,45211
unique,4
top,secondary
freq,23202


In [411]:
print(df['age'].corr(df['balance']))
print(df['day'].corr(df['campaign']))
print(df['day'].corr(df['pdays']))
print(df['pdays'].corr(df['previous']))

0.09778273937134754
0.16249021632619293
-0.09304407377294052
0.45481963548050075


In [412]:
df.y = (df.y == 'yes').astype(int)

In [413]:
from sklearn.model_selection import train_test_split

In [414]:
df_train_full, df_test = train_test_split(df, test_size=0.2, random_state=42)

In [415]:
df_train, df_val = train_test_split(df_train_full, test_size=0.33, random_state=42)

In [416]:
y_train = df_train.y.values
y_val = df_val.y.values
y_test = df_test.y.values

In [417]:
del df_train['y']
del df_val['y']
del df_test['y']
del df_train_full['y']

df_train = df_train.reset_index(drop=True)
df_val = df_val.reset_index(drop=True)
df_test = df_test.reset_index(drop=True)
df_train_full = df_train_full.reset_index(drop=True)

In [418]:
df_train.dtypes

,0
age,int64
job,object
marital,object
education,object
balance,int64
housing,object
contact,object
day,int64
month,object
duration,int64


In [419]:
categorical = ['job', 'marital', 'education', 'housing', 'contact', 'month',
               'poutcome']
numerical = ['age', 'balance', 'day', 'duration', 'campaign', 'pdays', 'previous']

In [420]:
from sklearn.metrics import mutual_info_score
from IPython.display import display

In [421]:
def calculate_mi(series):
    return mutual_info_score(series, y_train)
display(df_train[categorical].apply(calculate_mi).sort_values(ascending=False).to_frame('MI'))

,MI
poutcome,0.029391
month,0.025534
contact,0.012899
housing,0.010619
job,0.007187
education,0.002814
marital,0.002135


In [422]:
from sklearn.feature_extraction import DictVectorizer

In [423]:
train_dict = df_train[categorical + numerical].to_dict(orient='records')

In [424]:
dv = DictVectorizer(sparse=False)

In [425]:
X_train = dv.fit_transform(train_dict)

In [426]:
dv.get_feature_names_out()

array(['age', 'balance', 'campaign', 'contact=cellular',
       'contact=telephone', 'contact=unknown', 'day', 'duration',
       'education=primary', 'education=secondary', 'education=tertiary',
       'education=unknown', 'housing=no', 'housing=yes', 'job=admin.',
       'job=blue-collar', 'job=entrepreneur', 'job=housemaid',
       'job=management', 'job=retired', 'job=self-employed',
       'job=services', 'job=student', 'job=technician', 'job=unemployed',
       'job=unknown', 'marital=divorced', 'marital=married',
       'marital=single', 'month=apr', 'month=aug', 'month=dec',
       'month=feb', 'month=jan', 'month=jul', 'month=jun', 'month=mar',
       'month=may', 'month=nov', 'month=oct', 'month=sep', 'pdays',
       'poutcome=failure', 'poutcome=other', 'poutcome=success',
       'poutcome=unknown', 'previous'], dtype=object)

In [427]:
from sklearn.linear_model import LogisticRegression

In [428]:

  model = LogisticRegression(solver='liblinear', C=1.0, max_iter=1000, random_state=42)
  model.fit(X_train, y_train)
  val_dict = df_val[categorical + numerical].to_dict(orient='records')
  X_val = dv.fit_transform(val_dict)
  y_pred = model.predict_proba(X_val)[:, 1]
  y = y_pred >= 0.5
  accuracy = (y_val == y).mean()
  print(round(accuracy, 2))

0.9


In [429]:
for c in [0.01,0.1,1,10,100]:
  model = LogisticRegression(solver='liblinear', C=c, max_iter=1000, random_state=42)
  model.fit(X_train, y_train)
  val_dict = df_val[categorical + numerical].to_dict(orient='records')
  X_val = dv.fit_transform(val_dict)
  y_pred = model.predict_proba(X_val)[:, 1]
  y = y_pred >= 0.5
  accuracy = (y_val == y).mean()
  print(round(accuracy, 4))

0.8978
0.9005
0.9013
0.9014
0.9012


In [430]:
for f in features:
  print("remove ", f)
  dfa = df_train[features].copy()
  del dfa[f]
  dfb = df_val[features].copy()
  del dfb[f]
  train_dict = dfa.to_dict(orient='records')
  X_train = dv.fit_transform(train_dict)
  model.fit(X_train, y_train)
  val_dict = dfb.to_dict(orient='records')
  X_val = dv.fit_transform(val_dict)
  y_pred = model.predict_proba(X_val)[:, 1]
  y = y_pred >= 0.5
  accuracy_new = (y_val == y).mean()
  print(accuracy - accuracy_new)

remove  age
0.018766756032171594
remove  balance
0.018515415549597902
remove  marital
0.018766756032171594
remove  previous
0.017761394101876715
